# Day 14 Part1

In [3]:
import re

def simulate_robots(positions, velocities, seconds):
  """Simulates robot movement for a given number of seconds."""
  width = 101
  height = 103

  for _ in range(seconds):
    new_positions = []
    for p, v in zip(positions, velocities):
      x, y = map(int, re.findall(r'-?\d+', p))
      vx, vy = map(int, re.findall(r'-?\d+', v))

      x = (x + vx) % width  # Wrap around horizontally
      y = (y + vy) % height # Wrap around vertically

      new_positions.append(f"p={x},{y}")

    positions = new_positions

  return positions

def count_robots_in_quadrants(positions):
  """Counts robots in each quadrant."""
  quadrant_counts = [0, 0, 0, 0]
  for p in positions:
    x, y = map(int, re.findall(r'-?\d+', p))

    if x < 51 and y < 52:  # Quadrant 1 (top-left)
      quadrant_counts[0] += 1
    elif x >= 51 and y < 52: # Quadrant 2 (top-right)
      quadrant_counts[1] += 1
    elif x < 51 and y >= 52: # Quadrant 3 (bottom-left)
      quadrant_counts[2] += 1
    elif x >= 51 and y >= 52: # Quadrant 4 (bottom-right)
      quadrant_counts[3] += 1

  return quadrant_counts

def calculate_safety_factor(positions):
  """Calculates the safety factor."""
  quadrant_counts = count_robots_in_quadrants(positions)
  return quadrant_counts[0] * quadrant_counts[1] * \
         quadrant_counts[2] * quadrant_counts[3]

# Read robot data from input
with open("Input/InputDay14P1.txt", "r") as f:
  robot_data = [line.strip() for line in f]

positions = [re.findall(r"p=.*? ", line)[0] for line in robot_data]
velocities = [re.findall(r"v=.*?", line)[0] for line in robot_data]

# Simulate robot movement and calculate safety factor
final_positions = simulate_robots(positions, velocities, 100)
safety_factor = calculate_safety_factor(final_positions)

print("Safety Factor:", safety_factor)

ValueError: not enough values to unpack (expected 2, got 0)

In [4]:
def getRobots():
    with open("Input/InputDay14P1.txt") as file:
        return file.read()
    

# Formatting
robots = getRobots()
robots = [robot.split(" ") for robot in robots.split("\n")]
for i in range(len(robots)):
    robots[i][0] = list(reversed(list(map(int, robots[i][0].lstrip("p=").split(",")))))
    robots[i][1] = list(reversed(list(map(int, robots[i][1].lstrip("v=").split(",")))))

# Define size of grid
rows, cols = 103, 101
numSeconds = 100

# Loop through each robot
for i in range(len(robots)):
    # Move the robots by the number of spots moved per second * number of seconds
    # To account for pacman effect, % rows or cols to wrap around
    robots[i][0][0] = (robots[i][0][0] + (robots[i][1][0] * numSeconds)) % rows
    robots[i][0][1] = (robots[i][0][1] + (robots[i][1][1] * numSeconds)) % cols

# Get all robots in each quadrant
q1 = [x for x in robots if x[0][0] < rows//2 and x[0][1] < cols//2]
q2 = [x for x in robots if x[0][0] < rows//2 and x[0][1] > cols//2 and x[0][1] < cols]
q3 = [x for x in robots if x[0][0] > rows//2 and x[0][0] < rows and x[0][1] < cols//2]
q4 = [x for x in robots if x[0][0] > rows//2 and x[0][0] < rows and x[0][1] > cols//2 and x[0][1] < cols]

# Calculate and print the safety factor
print(f"Safety factor: {len(q1) * len(q2) * len(q3) * len(q4)}")


Safety factor: 233709840


# Part 2

In [5]:
def printGrid(robots, rows, cols):
    for i in range(rows):
        for j in range(cols):
            if [i,j] in [x[0] for x in robots]:
                print("*", end="")
            else:
                print(" ", end="")
        print()

# Breadth First Search every node 
# If there's a cluster larger than 20 robots then that's probably the tree
# 20 is just a guess at what I think would be a tell tale sign 
# of a tree made up of "most of the robots" would be
def checkIfChristmasTree(robotCoords: list[list[int]]) -> bool:

    # Loop through all coordinates
    for currentStartCoord in robotCoords:

        # Perform a breadth first search on each coordinate
        frontier = [currentStartCoord]
        explored = [currentStartCoord]
        clusterSize = 1
        while frontier:
            # Get coordinate from frontier
            currentLoc = frontier.pop(0)

            # Get surrounding points
            up = [currentLoc[0] - 1, currentLoc[1]]
            down = [currentLoc[0] + 1, currentLoc[1]]
            left = [currentLoc[0], currentLoc[1] - 1]
            right = [currentLoc[0], currentLoc[1] + 1]
            
            # Loop through each direction and if 
            for direction in [up, down, left, right]:
                if direction in robotCoords and direction not in explored:
                    frontier.append(direction)
                    explored.append(direction)

                    # Incriment the cluster size and check to see if it's big enough to be a tree
                    clusterSize += 1
                    if clusterSize >= 20:
                        return True

    # If we make it through every point then we did not find a tree
    return False

# Formatting
robots = getRobots()
robots = [robot.split(" ") for robot in robots.split("\n")]
for i in range(len(robots)):
    robots[i][0] = list(reversed(list(map(int, robots[i][0].lstrip("p=").split(",")))))
    robots[i][1] = list(reversed(list(map(int, robots[i][1].lstrip("v=").split(",")))))

# Define size of grid
rows, cols = 103, 101
seconds = 0

# Loop through each day
while True:
    seconds += 1
    # Loop through each robot
    for i in range(len(robots)):
        # Move the robots by the number of spots moved per second
        # To account for pacman effect, % rows or cols to wrap around
        robots[i][0][0] = (robots[i][0][0] + robots[i][1][0]) % rows
        robots[i][0][1] = (robots[i][0][1] + robots[i][1][1]) % cols

    # Check if there is a christmas tree formation
    if checkIfChristmasTree([x[0] for x in robots]):
        break

# Print out number of seconds until the easter egg and the grid
print(f"Seconds until easter egg: {seconds}")
printGrid(robots, rows, cols)

Seconds until easter egg: 6620
                                                                             *                       
                     *                                                   *                           
                                                                                                     
                                 *                                                                   
                       *  *                     *                                                    
                                                                                         *           
                                         *                                                           
                                                                                                     
                *              *                                                           *         
                                                   